In [1]:
import cluster as cl
import cluster_ts as cl_ts

import os
import numpy as np
import pandas as pd
import geopandas as gpd

from shapely import wkt
import fiona

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# basemap 
basemap_url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'

# centeral point of Samarra 
df_sites = pd.DataFrame({
    "longitude": [43.823543],
    "latitude": [34.340989],
    "name": ["Samarra Archaeological City"],
    "category": ["Cultural"],
    "date inscribed": ["2007"],
    "region": ["Arab States"],
    "url": ["https://whc.unesco.org/en/list/276"],
    "iso": [["IQ"]]
})

# Example Data

In [3]:
def create_subregions(chunks, sift_percentage_lon=0.5, sift_percentage_lat=0.415):
    """
        Shifts all polygons to 8 positions based on half the length in longitude and latitude, to create subregions - one for each direction from the center.

        Assumes aoi contains all similar polygons, and are similar to a square.

        Parameters:
            aoi (GeoDataFrame): The area of interest containing geometries.
            sift_percentage (float): Percentage of the length to shift the center point. Default is 0.5 (50%) for half the length.
    """

    if chunks.empty:
        return {}
    
    first_polygon = chunks.geometry.iloc[0]

    # take top two points of the polygon and get the length between them
    top_points = first_polygon.exterior.coords[:2]
    length_lon = abs(top_points[0][0] - top_points[1][0])

    # calculate the shift amount
    shift_amount_lon = length_lon * sift_percentage_lon 
    shift_amount_lat = length_lon * sift_percentage_lat

    shift_directions = {
        "left": (-shift_amount_lon, 0),
        "right": (shift_amount_lon, 0),
        "up": (0, shift_amount_lat),
        "down": (0, -shift_amount_lat),
        "top_left": (-shift_amount_lon, shift_amount_lat),
        "top_right": (shift_amount_lon, shift_amount_lat),
        "bottom_left": (-shift_amount_lon, -shift_amount_lat),
        "bottom_right": (shift_amount_lon, -shift_amount_lat),
    }

    subregions = []
    aoi = chunks.copy()
    aoi = aoi.dissolve()
        
    # Create subregions by shifting the geometries in all directions
    for _, (dx, dy) in shift_directions.items():
        gdf_shifted = chunks.copy()
        gdf_shifted["geometry"] = gdf_shifted["geometry"].translate(dx, dy)

        if subregions is not []:
            for region in subregions:
                for polygon in gdf_shifted.geometry:
                    for region_polygon in region.geometry:
                        if polygon.intersects(region_polygon):
                            # calculate % of intersection
                            intersection = polygon.intersection(region_polygon)
                            intersection_area = intersection.area
                            region_area = region_polygon.area
                            intersection_percentage = intersection_area / region_area if region_area > 0 else 0
                            
                            if intersection_percentage > 0.5: 
                                # if too high of an overlap skip
                                # print(f"Dropping polygon from subregion due to high overlap: {intersection_percentage:.2%} with existing region.")
                                gdf_shifted = gdf_shifted[gdf_shifted.geometry != polygon]
                                break         

        subregions.append(gdf_shifted)

    # Combine all into one GeoDataFrame
    subregions = pd.concat(subregions, ignore_index=True)

    # Reset index and return as GeoDataFrame
    subregions = subregions.reset_index(drop=True)

    return gpd.GeoDataFrame(subregions, crs=chunks.crs)

# Chunks of Samarra Archaeological City 
with fiona.open("chunks_new.shp") as src:
    chunks = gpd.GeoDataFrame.from_features(src, crs=src.crs)

# Create subregions based on the chunks
subregions = create_subregions(chunks)
subregions = pd.concat([subregions, chunks], ignore_index=True)

print(f"Subregions created: {len(subregions)}")

Subregions created: 35


In [6]:
mapping = {
    "NaN": 0,
    np.NaN: 0,
    "None": 0,
    "Water": 1,
    "Wetland": 2,
    "Agricultural": 3,
    "Urban": 4,
    "Wasteland": 5,
} 

data_dir = pd.DataFrame({
    "dir": [f"data/{year}.csv" for year in range(2019, 2025)],
    "year": [year for year in range(2019, 2025)]
})

labels = pd.read_csv("labels_backup_581.csv")
labels['geometry'] = labels['geometry'].apply(wkt.loads)
labels = gpd.GeoDataFrame(labels, geometry='geometry')
labels = labels.set_crs("EPSG:4326", allow_override=True)

In [7]:
from shapely.geometry import shape

def load_zones(dir):
    """
        Loads in polygons zones from a directory containing GeoJSON files.
    """

    files = []

    if dir is not [] and os.path.isfile(dir) and dir.endswith('.geojson'):
        files = [dir]
    else: 
        files = [f for f in os.listdir(dir) if f.endswith('.geojson')]
    polygons = []

    for file in files:
        with fiona.open(os.path.join(dir, file)) as src:
            for feature in src:
                polygons.append(shape(feature['geometry']))

    return polygons    

zones = load_zones("F:/0-Projects/GEE-UNESCO/COMM514_A_12_202425/zones")
boundary = load_zones("F:/0-Projects/GEE-UNESCO/COMM514_A_12_202425/zones/boundary")

boundary = gpd.GeoDataFrame(geometry=boundary, crs="EPSG:4326")
zones = gpd.GeoDataFrame(geometry=zones, crs="EPSG:4326")
# extract the geometry from the boundary
boundary_geom = boundary.geometry.values[0]

# Clustering 
Examples of how to load, save, fit and export

In [8]:
# SETUP the class, these you must save yourself if you wish to reload 
# subregion, mapping and data_dir are required, labels can be empty and loaded later 
# requires sequential years of data in data_dir between 2019 and 2024! else this will fail 
# you can use the original cl class to do this yourself if there are year gaps, 
#                                                   ...this class just streamlines the process 
clusterer = cl_ts.cluster_ts(
    ts_point_labels=labels,
    data_dir=data_dir,
    subregions=subregions,
    mapping=mapping,
    start_year=2019,
    end_year=2024,
    passes=3,
    )

#### instansiate_clusters then fit OR load your clusters

In [ ]:
# initialise the clusters (must be done or load the states)
clusterer.instansiate_clusters(
    cluster_class=cl.Cluster, 
    index_column="file_name"
)

In [ ]:
# clusterer.fit(save_state="product/gen_cluster581_15/cluster_")

In [ ]:
# clusterer.create_map(filename_prefix="product/gen_cluster581_15/cluster_")

#### load clusters

In [ ]:
# load clusters 
file_prefix = "product/gen_cluster900_15/cluster_"

# load the clusters from the file
#   this still requires the setup of the clusterer but no instansiation required
clusterer.load_states(filename_prefix=file_prefix)

Instantiating cluster for year 2019...
Instantiating cluster for year 2020...
Instantiating cluster for year 2021...
Instantiating cluster for year 2022...
Instantiating cluster for year 2023...
Instantiating cluster for year 2024...


In [ ]:
clusterer.cluster_list[0].sparse_matrix.head()

# Updating Labels
This is an example using GeeMap, but this is just one way todo it, you can create the with the software/libaries of choice, they just have to fit the year_start, year_end, point format and be a GeoDataframe 

In [ ]:
import ee
import geemap
import ipywidgets as widgets
from shapely.geometry import Point
import pandas as pd 

ee.Authenticate()
ee.Initialize(project="jameswilliamchamberlain")

In [ ]:
# 
label_list = ['Urban', 'Agricultural', 'Water', 'Wasteland', 'Wetland', 'Other']
date_range = [i for i in range(2019, 2025)] 

collection_name = "COPERNICUS/S2_SR_HARMONIZED"

with fiona.open("aoi.geojson") as src:
    aoi = gpd.GeoDataFrame.from_features(src, crs=src.crs)

In [ ]:
# util functions 
def collect_points_from_geemap(map_obj, label):
    """
    Collect all drawn point features from a geemap.Map that uses ee.Feature objects,
    and return them as a GeoDataFrame with a label.

    Parameters:
        map_obj (geemap.Map): The interactive map.
        label (str): Label to assign to all collected points.

    Returns:
        GeoDataFrame: With geometry and 'label' columns.
    """

    features = map_obj.draw_features

    if not features:
        return gpd.GeoDataFrame(columns=["geometry", "label"], geometry="geometry")

    points = []
    for f in features:
        try:
            geom = f.geometry()  # call the method
            if geom.getInfo()["type"] == "Point":
                coords = geom.coordinates().getInfo()  # [lon, lat]
                points.append(Point(coords))
        except Exception as e:
            print("Skipping feature due to error:", e)

    if not points:
        return gpd.GeoDataFrame(columns=["geometry", "label"], geometry="geometry")

    gdf = gpd.GeoDataFrame(geometry=points)
    gdf["label"] = label
    gdf.set_crs("EPSG:4326", inplace=True)
    return gdf

def gen_basemap(basemap_url=None, aoi=gpd.GeoDataFrame(), polygons=None):
    """
        Generates a basemap with the specified URL and adds polygons if provided.

        Parameters:
            basemap_url (str): The URL of the basemap to be used.
            polygons (GeoDataFrame, optional): Polygons to be added to the map. Defaults to None.
            aoi (GeoDataFrame): The area of interest to be displayed on the map. (can be a set of polygons or a single polygon)
        
        Returns:
            geemap.Map: A geemap map object with the specified basemap and polygons.
    """
    m = geemap.Map()

    if not basemap_url:
        basemap_url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'
        
    m.add_basemap(basemap_url, name="Google Satellite", attribution="Google")

    # center on aoi
    if not aoi.empty:
        m.add_gdf(aoi, layer_name="AOI", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})
        center_points = aoi.geometry.unary_union.centroid.coords[0]
        m.setCenter(center_points[0], center_points[1], 10)
    
    if polygons is not None:
        m.add_gdf(polygons, layer_name="Polygons", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})
    
    return m

In [ ]:
# clear labels 
labels = gpd.GeoDataFrame(columns=["label", "geometry", "start_year", "end_year"], geometry="geometry")

In [ ]:
m = geemap.Map()

def map_polygon(polygon, collection_name, layer_name, yyyymmdd1="2024-01-01", yyyymmdd2="2024-12-29", num_tasks=10):

    # Load in Year Data to Aid in Label Creation
    collection = ee.ImageCollection(collection_name) \
        .filterDate(yyyymmdd1, yyyymmdd2) \
        .filterBounds(polygon) \
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20)) \
        .median() \
        .clip(polygon)
    
    vis = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

    m.addLayer(collection, vis, layer_name)

    # center map on the polygon
    coords = polygon.geometry().centroid().coordinates().getInfo()
    m.setCenter(coords[0], coords[1], 10)

label_dropdown = widgets.Dropdown(
    options=label_list,
    description='Label:',
    value=label_list[0],
)

# Plots Each Year
for year in date_range:
    map_polygon(geemap.geopandas_to_ee(aoi), collection_name, f"Year {year}", yyyymmdd1=f"{year}-01-01", yyyymmdd2=f"{year}-12-29")

date_slider = widgets.SelectionSlider(
    options=[str(year) for year in range(2019, 2025)], # 2019 to 2024 for full years
    value='2019',
    description='Date:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

time_frame_slider = widgets.SelectionSlider(
    options=[str(month) for month in range(1, 13)], # default 12 as its the full year 
    value='3',
    description='timeframe:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

colour_dict = {
    "Urban": "blue",
    "Agricultural": "green",
    "Water": "cyan",
    "Wasteland": "brown",
    "Wetland": "lightblue", 
} # other is grey by default

def append_label_to_points(b):
    """Appends the selected label to the points in the labels GeoDataFrame."""

    global labels

    selected_label = label_dropdown.value
    start_year = int(date_slider.value)
    span = int(time_frame_slider.value)
    end_year = start_year + span - 1

    # Extract drawn features (assumed to be Points)
    all_drawn_features = m.draw_features
    if not all_drawn_features:
        print("No features drawn.")
        return
    
    points = []
    for feature in all_drawn_features:
        try:
            coords = feature.geometry().coordinates().getInfo()  # [lon, lat]
            point = Point(coords)
            points.append(point)
        except Exception as e:
            print("Skipping feature due to error:", e)

    # create dataframe from points
    new_labels = gpd.GeoDataFrame({
        'label': [selected_label] * len(points),
        'start_year': [start_year] * len(points),
        'end_year': [end_year] * len(points),
        'geometry': points
    }, crs="EPSG:4326")

    # Clear All Drawn Features
    m.draw_control.clear()


    # Append to the existing labels GeoDataFrame
    labels = pd.concat([labels, new_labels], ignore_index=True)
    

# Label Range  
label_output = widgets.Label()
def update_label(*args):
    start_year = int(date_slider.value)
    span = int(time_frame_slider.value)
    end_year = start_year + span - 1
    label_output.value = f"Selected range: {start_year} to {end_year}"

date_slider.observe(update_label, names='value')
time_frame_slider.observe(update_label, names='value')

update_label() 

append_btn = widgets.Button(description="Append Label to Points")
append_btn.on_click(append_label_to_points)

# display(widgets.HBox([date_slider, month_slider, time_frame_slider, render_btn]))
display(widgets.VBox([
    widgets.HBox([date_slider, time_frame_slider]),
    widgets.HBox([label_dropdown, label_output, append_btn]),
]))

m

In [ ]:
# add from geodataframe
def add_gdf_to_map(gdf, label, colour="Red", fillOpacity=0.5):
    if gdf.empty:
        print("GeoDataFrame is empty, nothing to add.")
        return

    m.add_gdf(gdf, layer_name=label, style={"color": colour, "fillColor": colour, "fillOpacity": fillOpacity})

add_gdf_to_map(zones, "Zones", colour="Red", fillOpacity=0.25)
add_gdf_to_map(boundary, "Boundary", colour="Yellow", fillOpacity=0.0)

In [ ]:
# labels = gpd.GeoDataFrame(columns=["label", "geometry", "start_year", "end_year"], geometry="geometry") # rest list (code left for ease of access)
labels

In [ ]:
labels.to_csv("labels_temp_319.csv", index=False)

In [ ]:
temp_labels = pd.read_csv("labels_temp.csv")

In [ ]:
len(temp_labels)

# Update Labels and Review New Map

In [ ]:
clusterer.update_labels(labels)

In [ ]:
# save
clusterer.save_states(filename_prefix="product/gen_cluster900_15/cluster_")

In [ ]:
clusterer.create_map(filename_prefix="product/gen_cluster900_15/cluster_")

# Analysis 

In [ ]:
# load predictions old 
clusterer.cluster_list[0].predictions = pd.read_csv("cluster581/cluster581_2019_predictions.csv").set_index('file_name')
clusterer.cluster_list[1].predictions = pd.read_csv("cluster581/cluster581_2020_predictions.csv").set_index('file_name')
clusterer.cluster_list[2].predictions = pd.read_csv("cluster581/cluster581_2021_predictions.csv").set_index('file_name')
clusterer.cluster_list[3].predictions = pd.read_csv("cluster581/cluster581_2022_predictions.csv").set_index('file_name')
clusterer.cluster_list[4].predictions = pd.read_csv("cluster581/cluster581_2023_predictions.csv").set_index('file_name')
clusterer.cluster_list[5].predictions = pd.read_csv("cluster581/cluster581_2023_predictions.csv").set_index('file_name')

In [ ]:
clusterer.cluster_list[5].predictions 

In [ ]:
# collect the prediction columns from the clusters
# predictions = clusterer.collect_predictions()

def collect_predictions_by_year(clusterer):
    all_year_dfs = []  # To store each year's predictions
    all_geos = pd.Series(dtype=object)  # To collect the most complete .geo values

    for i, cluster in enumerate(clusterer.cluster_list):
        year = clusterer.start_year + i
        df = cluster.predictions[['predicted_label', '.geo']].copy()

        # Rename prediction column to the year
        df = df.rename(columns={'predicted_label': str(year)})

        # Add to our list of yearly prediction DataFrames
        all_year_dfs.append(df[[str(year)]])

        # Combine geo info for this cluster
        all_geos = all_geos.combine_first(df['.geo'])

    # Outer join on index to merge all yearly predictions
    merged = pd.concat(all_year_dfs, axis=1)

    # Add the unified .geo column
    merged['.geo'] = all_geos

    return merged
    # for cluster in clusterer.cluster_list:
    #     predictions = cluster.create_predictions() 
    #     predictions.append(cluster.predictions['labels'].values)
    
    # return pd.DataFrame(predictions)

merged = collect_predictions_by_year(clusterer)

In [ ]:
# save merged predictions
# merged.to_csv("product/gen_cluster900_15/merged_predictions_15_900.csv", index=True)

In [ ]:
merged = pd.read_csv("product/gen_cluster900_15/merged_predictions_15_900.csv", index_col=0)

In [ ]:
merged

In [ ]:
boundary_geom = gpd.GeoDataFrame(geometry=boundary, crs="EPSG:4326")

In [ ]:
# clip to region 
import utils as ut

gdf_in, gdf_out = ut.clip_to_polygon(boundary_geom, merged)
print(len(gdf_in), len(gdf_out))

In [ ]:
gdf_in.drop(columns=['file_name'], inplace=True)

In [ ]:
gdf_in

In [ ]:
# drop duplicates based on index
gdf_in = gdf_in.drop_duplicates(subset=['.geo'])

In [ ]:
class_totals_per_year = gdf_in.iloc[:, :6].apply(pd.Series.value_counts)
class_totals_per_year

In [ ]:
class_totals_per_year = class_totals_per_year.sort_index()

# Transpose: years on x-axis, classes as lines
class_totals_per_year.T.plot(kind='line', marker='o', figsize=(10, 6))

plt.title("Land Cover Class Changes Over Time")
plt.xlabel("Year")
plt.ylabel("Number of Tiles")
plt.grid(True)
plt.legend(title="Class", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# create a list of land use per year (totals of unique land use classes)



In [ ]:
land_use_per_year

In [ ]:
import matplotlib.pyplot as plt

years = merged.columns[:-1]  

# plot totals 
for year in years:
    plt.plot(gdf_in[year].value_counts().index, merged[year].value_counts().values, label=year)
plt.xlabel('Predicted Label')
plt.ylabel('Count')
plt.title('Predicted Labels Count')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

years = gdf_in.columns[:-1]  

# plot totals 
for year in years:
    plt.plot(gdf_in[year].value_counts().index, gdf_in[year].value_counts().values, label=year)
plt.xlabel('Predicted Label')
plt.ylabel('Count')
plt.title('Predicted Labels Count')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

years = gdf_in.columns[:-1]  

proportions = {}
for year in years:
    year_counts = gdf_in[year].value_counts(normalize=True)
    proportions[year] = year_counts

proportion_df = pd.DataFrame(proportions).fillna(0)

plt.figure(figsize=(12, 6))

for label in proportion_df.index:
    plt.plot(proportion_df.columns, proportion_df.loc[label], marker='o', label=label)

plt.title('Proportion of Classes')
plt.xlabel('Year')
plt.ylabel('Proportion')
plt.xticks(rotation=45)
plt.legend(title='Class', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# import matplotlib.pyplot as plt

# years = gdf_in.columns[:-1]  

# proportions = {}
# for year in years:
#     year_counts = gdf_in[year].value_counts(normalize=True)
#     proportions[year] = year_counts

# proportion_df = pd.DataFrame(proportions).fillna(0)

# plt.figure(figsize=(12, 6))

# for label in proportion_df.index:
#     plt.plot(proportion_df.columns, proportion_df.loc[label], marker='o', label=label)

# plt.title('Proportion of Classes')
# plt.xlabel('Year')
# plt.ylabel('Proportion')
# plt.xticks(rotation=45)
# plt.legend(title='Class', bbox_to_anchor=(1.05, 1), loc='upper left')
# plt.tight_layout()
# plt.show()

In [ ]:
# 2019 
data_2019 = clusterer.cluster_list[0].data
labels_2019 = clusterer.cluster_list[0].predictions

In [ ]:
# data_2019
urban_ids = labels_2019[labels_2019['predicted_label'] == "Urban"].index
urban_data_2019 = data_2019.loc[data_2019.index.isin(urban_ids)]
urban_data_2019_summary = ut.summarise(urban_data_2019)
ut.create_figs(urban_data_2019_summary)

In [ ]:
# agriculture 
agriculture_ids = labels_2019[labels_2019['predicted_label'] == "Agricultural"].index
agriculture_data_2019 = data_2019.loc[data_2019.index.isin(agriculture_ids)]
agriculture_data_2019_summary = ut.summarise(agriculture_data_2019)
ut.create_figs(agriculture_data_2019_summary)

In [ ]:
# urban_vs_agriculture = agriculture_data_2019_summary - urban_data_2019_summary 
urban_vs_agriculture = urban_data_2019_summary - agriculture_data_2019_summary
urban_vs_agriculture = urban_vs_agriculture.abs()
ut.create_figs(urban_vs_agriculture)

In [ ]:
# urban_vs_agriculture = agriculture_data_2019_summary - urban_data_2019_summary 
urban_vs_agriculture = agriculture_data_2019_summary - urban_data_2019_summary 
ut.create_figs(urban_vs_agriculture)

In [ ]:
background_values = data_2019.copy()
backgorund_summary = ut.summarise(background_values)
ut.create_figs(backgorund_summary)

In [ ]:
gdf_in_2019, gdf_out_2019 = ut.clip_to_polygon(boundary_geom, data_2019.copy())
print(len(gdf_in_2019), len(gdf_out_2019))

In [ ]:
background_values = gdf_in_2019.copy()
backgorund_summary = ut.summarise(background_values)
ut.create_figs(backgorund_summary)